In [ ]:
#!pip install pycaret

In [29]:
import pandas as pd
import pycaret
from pycaret.classification import *
import numpy as np
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

In [30]:
Offline_data = pd.read_csv('D:/NIDA/DADS6005/Finalproject/rev07_EDA/Offline_data_use.csv')
Offline_data = Offline_data.drop(columns=['Unnamed: 0'])
print(Offline_data.head())

             timestamp    TP2    TP3     H1  DV_pressure  Reservoirs  \
0  2020-04-15 01:57:18 -0.014  9.654  9.640       -0.022       9.654   
1  2020-06-15 18:54:50 -0.012  8.284  8.272       -0.020       8.284   
2  2020-03-23 09:40:00 -0.018  8.070  8.062       -0.024       8.072   
3  2020-08-15 07:04:07 -0.012  9.142  9.130       -0.022       9.144   
4  2020-04-03 11:57:28 -0.012  9.390  9.378       -0.020       9.392   

   Oil_temperature  Motor_current  COMP  DV_eletric  Towers  MPG  LPS  \
0           61.675         0.0500   1.0         0.0     1.0  1.0  0.0   
1           62.125         0.0400   1.0         0.0     1.0  1.0  0.0   
2           51.950         0.0400   1.0         0.0     1.0  1.0  0.0   
3           63.775         0.0425   1.0         0.0     1.0  1.0  0.0   
4           64.675         0.0425   1.0         0.0     1.0  1.0  0.0   

   Pressure_switch  Oil_level  Caudal_impulses Severity  
0              1.0        1.0              1.0      low  
1           

In [31]:
# Create the DataFrame
df = pd.DataFrame(Offline_data)

# Convert 'timestamp' to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Set 'timestamp' as the index
df.set_index('timestamp', inplace=True)
print(df.head())

                       TP2    TP3     H1  DV_pressure  Reservoirs  \
timestamp                                                           
2020-04-15 01:57:18 -0.014  9.654  9.640       -0.022       9.654   
2020-06-15 18:54:50 -0.012  8.284  8.272       -0.020       8.284   
2020-03-23 09:40:00 -0.018  8.070  8.062       -0.024       8.072   
2020-08-15 07:04:07 -0.012  9.142  9.130       -0.022       9.144   
2020-04-03 11:57:28 -0.012  9.390  9.378       -0.020       9.392   

                     Oil_temperature  Motor_current  COMP  DV_eletric  Towers  \
timestamp                                                                       
2020-04-15 01:57:18           61.675         0.0500   1.0         0.0     1.0   
2020-06-15 18:54:50           62.125         0.0400   1.0         0.0     1.0   
2020-03-23 09:40:00           51.950         0.0400   1.0         0.0     1.0   
2020-08-15 07:04:07           63.775         0.0425   1.0         0.0     1.0   
2020-04-03 11:57:28           

In [32]:
X, y = make_classification(n_classes=3, class_sep=2, weights=[0.01, 0.03, 0.96],
                           n_informative=3, n_redundant=1, flip_y=0,
                           n_features=20, n_clusters_per_class=1,
                           n_samples=100000, random_state=10)
 
print('Original dataset shape %s' % Counter(y))
 
# Define the sampling strategy
# Over-sampling strategy: set to a number larger than the original minority class sizes
over = SMOTE(sampling_strategy={0: 28032, 1: 28032})
 
# Under-sampling strategy: match the smallest minority class
under = RandomUnderSampler(sampling_strategy={2: 28032})
 
# Create the pipeline
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)
 
# Apply hybrid sampling
X_resampled, y_resampled = pipeline.fit_resample(X, y)
 
print('Resampled dataset shape %s' % Counter(y_resampled))

Original dataset shape Counter({2: 96000, 1: 3000, 0: 1000})
Resampled dataset shape Counter({0: 28032, 1: 28032, 2: 28032})


In [33]:
y_resampled = pd.Series(y_resampled).map({0: 'low', 1: 'medium', 2: 'high'})

# Print the resampled dataset shape with categorical labels
print('Resampled dataset shape %s' % Counter(y_resampled))

Resampled dataset shape Counter({'low': 28032, 'medium': 28032, 'high': 28032})


In [34]:
# Create a DataFrame with X_resampled and y_resampled
resampled_data = pd.DataFrame(X_resampled, columns=[f'feature_{i}' for i in range(X_resampled.shape[1])])
resampled_data['Severity'] = y_resampled

# Print the resampled dataset shape with categorical labels
print('Resampled dataset shape %s' % Counter(y_resampled))
print(resampled_data.head()) 

Resampled dataset shape Counter({'low': 28032, 'medium': 28032, 'high': 28032})
   feature_0  feature_1  feature_2  feature_3  feature_4  feature_5  \
0  -2.704478  -0.889735   0.837853  -0.585085   1.191450  -2.296469   
1  -2.126720  -3.787947   0.632790  -0.409219   3.837749   0.279838   
2  -3.075743  -1.310555  -0.421874   0.257948   1.089708   1.016111   
3  -4.114018  -1.914137   0.293514   0.344696   1.777276  -1.414871   
4   0.060580  -2.832238  -0.512871   0.197951   2.886265   0.305603   

   feature_6  feature_7  feature_8  feature_9  ...  feature_11  feature_12  \
0   0.622404   0.074794  -2.563294   0.566078  ...   -0.057669    0.277114   
1   0.419981  -0.565131  -0.348729   1.473602  ...    2.173111   -1.397744   
2  -0.637232   0.097701  -0.642069   0.462926  ...    0.650254   -0.755835   
3   0.931169   0.075613  -1.093738  -1.462616  ...   -1.233206   -1.163277   
4   0.358214  -0.609193   0.217892  -0.400077  ...   -0.334955   -0.998577   

   feature_13  feature_1

In [35]:
resampled_data.info

<bound method DataFrame.info of        feature_0  feature_1  feature_2  feature_3  feature_4  feature_5  \
0      -2.704478  -0.889735   0.837853  -0.585085   1.191450  -2.296469   
1      -2.126720  -3.787947   0.632790  -0.409219   3.837749   0.279838   
2      -3.075743  -1.310555  -0.421874   0.257948   1.089708   1.016111   
3      -4.114018  -1.914137   0.293514   0.344696   1.777276  -1.414871   
4       0.060580  -2.832238  -0.512871   0.197951   2.886265   0.305603   
...          ...        ...        ...        ...        ...        ...   
84091   0.767391  -1.994388   1.152330  -0.910293   3.086917   0.531667   
84092   1.399459  -2.410230   0.412240  -0.544187   2.025398   0.016784   
84093   1.360190  -3.283492   1.123743   0.947630   0.518480   0.507642   
84094   2.445298  -0.493102   1.585614  -1.579327   2.932698   0.263638   
84095   0.447362  -2.611833  -0.228141   0.565499   2.701921   0.145251   

       feature_6  feature_7  feature_8  feature_9  ...  feature_11 

In [36]:
# Use PyCaret for classification
clf = setup(resampled_data, target='Severity', session_id = 123)

,Description,Value
0,Session id,123
1,Target,Severity
2,Target type,Multiclass
3,Target mapping,"high: 0, low: 1, medium: 2"
4,Original data shape,"(84096, 21)"
5,Transformed data shape,"(84096, 21)"
6,Transformed train set shape,"(58867, 21)"
7,Transformed test set shape,"(25229, 21)"
8,Numeric features,20
9,Preprocess,True


In [37]:
# functional API
best = clf.compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.9985,1.0000,0.9985,0.9985,0.9985,0.9978,0.9978,0.9150
lightgbm,Light Gradient Boosting Machine,0.9983,1.0000,0.9983,0.9983,0.9983,0.9975,0.9975,1.8200
rf,Random Forest Classifier,0.9976,1.0000,0.9976,0.9976,0.9976,0.9964,0.9964,3.6760
dt,Decision Tree Classifier,0.9948,0.9961,0.9948,0.9948,0.9948,0.9921,0.9921,0.6110
gbc,Gradient Boosting Classifier,0.9942,0.0000,0.9942,0.9942,0.9942,0.9913,0.9913,43.6880
knn,K Neighbors Classifier,0.9926,0.9979,0.9926,0.9928,0.9926,0.9889,0.9890,1.7160
qda,Quadratic Discriminant Analysis,0.9924,0.0000,0.9924,0.9924,0.9924,0.9885,0.9886,0.1200
lr,Logistic Regression,0.9832,0.0000,0.9832,0.9833,0.9832,0.9748,0.9749,0.7900
svm,SVM - Linear Kernel,0.9811,0.0000,0.9811,0.9815,0.9812,0.9717,0.9719,0.1500
nb,Naive Bayes,0.9604,0.9942,0.9604,0.9609,0.9606,0.9406,0.9407,0.1140


Processing:   0%|          | 0/61 [00:00<?, ?it/s]

In [38]:
evaluate_model(best)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [39]:
predict_model(best)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extra Trees Classifier,0.9983,1.0000,0.9983,0.9983,0.9983,0.9974,0.9974


,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,Severity,prediction_label,prediction_score
34404,-0.952028,3.190894,-1.731153,-1.835356,2.370221,-0.171084,0.252558,0.033368,-0.232687,-0.215533,...,-0.313636,-0.209332,0.425925,0.255882,1.209986,0.686666,1.561828,medium,medium,1.00
43947,-2.869423,1.853954,0.519723,-1.234367,2.663659,0.896910,0.187766,0.106120,-1.198916,-1.306671,...,-0.342382,0.815445,-0.364431,-0.603112,0.607370,-0.237543,0.405574,medium,medium,1.00
52341,-2.392855,1.129043,-0.276195,-1.292305,1.180516,0.665659,-0.235953,0.195772,-0.659171,-0.915779,...,0.339690,-1.200966,-0.990953,-0.796207,-0.374539,0.954537,0.739412,medium,medium,1.00
40211,-2.282001,2.563775,-1.496729,0.223638,2.131461,-0.610497,-0.205485,0.165921,-0.350869,-1.500903,...,0.346646,-1.298507,0.572819,-0.217547,-0.460465,-1.956137,0.593224,medium,medium,1.00
32848,-2.826583,1.362384,1.267395,0.008679,1.894391,-0.323848,1.337517,0.166612,-0.135411,0.356329,...,0.170006,-0.308446,0.381327,0.458690,0.056321,-0.191576,-0.285535,medium,medium,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63987,2.004009,-3.505821,0.909756,0.809560,0.058990,1.864974,0.033347,-0.499760,0.042638,-0.383310,...,0.961974,-1.685303,-0.074006,1.361166,-1.428002,0.143065,1.059216,high,high,1.00
61936,0.806855,-2.531347,2.042807,0.020063,1.356889,0.744991,-1.341664,-0.465069,-0.353156,-1.833560,...,0.596017,0.652290,-1.731677,-0.689395,0.043670,-1.058194,-1.408629,high,high,0.98
1531,-3.064174,-0.949993,-0.097158,0.210234,0.801251,-0.563965,0.488824,0.162541,-0.009115,0.514090,...,-0.105275,-0.672287,0.997537,-0.482920,1.422433,2.315657,0.376166,low,low,1.00
51112,-2.867111,1.733357,0.873043,-0.995198,2.264881,-0.029863,-0.294557,0.149962,0.850779,1.011773,...,-1.634595,-0.573749,0.390326,-0.152364,1.623851,-0.204711,-0.232614,medium,medium,1.00


In [40]:
# Save the best model
save_model(best, 'Model_et')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('label_encoding',
                  TransformerWrapperWithInverse(exclude=None, include=None,
                                                transformer=LabelEncoder())),
                 ('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['feature_0', 'feature_1',
                                              'feature_2', 'feature_3',
                                              'feature_4', 'feature_5',
                                              'feature_6', 'feature_7',
                                              'feature_8', 'feature_9',
                                              'feature_10', 'featur...
                  ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0,
                                       class_weight=None, criterion='gini',
                                       max_depth=None, max_features='sqrt',
                          

In [41]:
# Save lightgbm
lightgbm = create_model('lightgbm')
save_model(lightgbm, 'Model_lightgbm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9983,1.0000,0.9983,0.9983,0.9983,0.9975,0.9975
1,0.9986,0.9999,0.9986,0.9986,0.9986,0.9980,0.9980
2,0.9990,1.0000,0.9990,0.9990,0.9990,0.9985,0.9985
3,0.9981,1.0000,0.9981,0.9981,0.9981,0.9972,0.9972
4,0.9985,1.0000,0.9985,0.9985,0.9985,0.9977,0.9977
5,0.9976,1.0000,0.9976,0.9976,0.9976,0.9964,0.9964
6,0.9986,1.0000,0.9986,0.9986,0.9986,0.9980,0.9980
7,0.9978,1.0000,0.9978,0.9978,0.9978,0.9967,0.9967
8,0.9986,1.0000,0.9986,0.9986,0.9986,0.9980,0.9980


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('label_encoding',
                  TransformerWrapperWithInverse(exclude=None, include=None,
                                                transformer=LabelEncoder())),
                 ('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['feature_0', 'feature_1',
                                              'feature_2', 'feature_3',
                                              'feature_4', 'feature_5',
                                              'feature_6', 'feature_7',
                                              'feature_8', 'feature_9',
                                              'feature_10', 'featur...
                  LGBMClassifier(boosting_type='gbdt', class_weight=None,
                                 colsample_bytree=1.0, importance_type='split',
                                 learning_rate=0.1, max_depth=-1,
                             

In [42]:
# Save Random Forest Classifier
rf = create_model('rf')
save_model(rf, 'Model_rf')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9969,1.0000,0.9969,0.9970,0.9969,0.9954,0.9954
1,0.9978,1.0000,0.9978,0.9978,0.9978,0.9967,0.9967
2,0.9986,1.0000,0.9986,0.9986,0.9986,0.9980,0.9980
3,0.9973,1.0000,0.9973,0.9973,0.9973,0.9959,0.9959
4,0.9981,1.0000,0.9981,0.9981,0.9981,0.9972,0.9972
5,0.9976,1.0000,0.9976,0.9976,0.9976,0.9964,0.9964
6,0.9975,1.0000,0.9975,0.9975,0.9975,0.9962,0.9962
7,0.9969,1.0000,0.9969,0.9970,0.9969,0.9954,0.9954
8,0.9978,1.0000,0.9978,0.9978,0.9978,0.9967,0.9967


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('label_encoding',
                  TransformerWrapperWithInverse(exclude=None, include=None,
                                                transformer=LabelEncoder())),
                 ('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['feature_0', 'feature_1',
                                              'feature_2', 'feature_3',
                                              'feature_4', 'feature_5',
                                              'feature_6', 'feature_7',
                                              'feature_8', 'feature_9',
                                              'feature_10', 'featur...
                  RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                         class_weight=None, criterion='gini',
                                         max_depth=None, max_features='sqrt',
                     

In [43]:
new_data = resampled_data.copy()
new_data.drop('Severity', axis=1, inplace=True)
new_data.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19
0,-2.704478,-0.889735,0.837853,-0.585085,1.191450,-2.296469,0.622404,0.074794,-2.563294,0.566078,-0.034451,-0.057669,0.277114,-0.203399,0.178404,-1.303323,-0.697987,0.957400,-0.872940,0.860297
1,-2.126720,-3.787947,0.632790,-0.409219,3.837749,0.279838,0.419981,-0.565131,-0.348729,1.473602,1.269339,2.173111,-1.397744,0.889151,1.334136,0.499608,1.212155,0.209807,-0.238219,-0.601408
2,-3.075743,-1.310555,-0.421874,0.257948,1.089708,1.016111,-0.637232,0.097701,-0.642069,0.462926,-0.908920,0.650254,-0.755835,-0.123968,-2.343253,-0.765385,-1.001643,0.582796,-1.659497,1.049993
3,-4.114018,-1.914137,0.293514,0.344696,1.777276,-1.414871,0.931169,0.075613,-1.093738,-1.462616,-0.795923,-1.233206,-1.163277,-0.859882,-1.567692,0.642154,0.648344,-0.353052,-2.593154,0.680832
4,0.060580,-2.832238,-0.512871,0.197951,2.886265,0.305603,0.358214,-0.609193,0.217892,-0.400077,-0.534183,-0.334955,-0.998577,1.485259,0.899876,0.421408,-0.348277,0.536416,1.154489,-1.024489


In [44]:
predictions = predict_model(best, data = new_data)
predictions.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,prediction_label,prediction_score
0,-2.704478,-0.889735,0.837853,-0.585085,1.191450,-2.296469,0.622404,0.074794,-2.563294,0.566078,...,0.277114,-0.203399,0.178404,-1.303324,-0.697987,0.957400,-0.872940,0.860297,low,1.00
1,-2.126720,-3.787947,0.632790,-0.409219,3.837749,0.279838,0.419981,-0.565131,-0.348729,1.473602,...,-1.397744,0.889151,1.334136,0.499608,1.212155,0.209807,-0.238219,-0.601408,low,1.00
2,-3.075742,-1.310555,-0.421874,0.257948,1.089707,1.016111,-0.637232,0.097701,-0.642069,0.462926,...,-0.755835,-0.123968,-2.343253,-0.765385,-1.001643,0.582796,-1.659497,1.049994,low,1.00
3,-4.114017,-1.914137,0.293514,0.344696,1.777276,-1.414871,0.931169,0.075613,-1.093738,-1.462616,...,-1.163277,-0.859882,-1.567692,0.642154,0.648344,-0.353052,-2.593154,0.680832,low,0.99
4,0.060580,-2.832238,-0.512871,0.197951,2.886265,0.305603,0.358214,-0.609193,0.217892,-0.400077,...,-0.998577,1.485260,0.899876,0.421408,-0.348277,0.536416,1.154489,-1.024489,low,1.00
